In [4]:
import pandas as pd
import os

# CSV 파일 경로 (직접 경로로 바꿔주세요)
csv_path = "data/test_with_answer.csv"

# 출력 디렉토리 설정
output_dir = "docs/train_chunks"
os.makedirs(output_dir, exist_ok=True)

# 몇 행씩 나눌 것인지 설정
chunk_size = 500  # 필요에 따라 100, 1000 등으로 조정 가능

# CSV 불러오기
df = pd.read_csv(csv_path)

# 총 분할 수 계산
num_chunks = (len(df) + chunk_size - 1) // chunk_size

# 각 분할을 텍스트로 저장
for i in range(num_chunks):
    chunk = df.iloc[i * chunk_size : (i + 1) * chunk_size]
    lines = []
    for _, row in chunk.iterrows():
        err = row.get("err_sentence", "")
        cor = row.get("cor_sentence", "")
        if pd.notnull(err) and pd.notnull(cor):
            lines.append(f"입력: {err}\n출력: {cor}\n")

    file_path = os.path.join(output_dir, f"train_chunk_{i+1:02}.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

print(f"{num_chunks}개의 파일이 '{output_dir}'에 저장되었습니다.")

22개의 파일이 'docs/train_chunks'에 저장되었습니다.


In [1]:
import asyncio
import pandas as pd
import json
from tqdm import tqdm
from itertools import cycle
from prompts.template_format import format_prompt
from optimizer.evaluator import compute_scores
from openai import AsyncOpenAI
from dotenv import load_dotenv
import nest_asyncio
import os

# 🧠 비동기 이벤트 루프 충돌 방지 (Jupyter 전용)
nest_asyncio.apply()
load_dotenv()

# 📦 API 키 최대 10개 로딩
API_KEYS = [os.getenv(f"UPSTAGE_API_KEY_{i}") for i in range(1, 11)]
API_KEYS = [k for k in API_KEYS if k]

clients = [
    AsyncOpenAI(api_key=key, base_url="https://api.upstage.ai/v1")
    for key in API_KEYS
]
client_cycle = cycle(clients)
semaphores = {client: asyncio.Semaphore(3) for client in clients}  # 키당 최대 동시 3건 제한

# 📄 템플릿 로드
graph_path = "data/prompt_graph.jsonl"
selected_template_id = "base_02"
with open(graph_path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        if obj.get("id") == selected_template_id:
            template_str = obj["template"]
            break

# 🧪 데이터 로딩
df = pd.read_csv("data/test_with_answer.csv")
sample = [
    {"input": row["err_sentence"], "output": row["cor_sentence"]}
    for _, row in df.iterrows()
    if pd.notnull(row["err_sentence"]) and pd.notnull(row["cor_sentence"])
]

# 🚀 제한 포함된 LLM 호출
async def call_with_limit(client, messages, input_text):
    async with semaphores[client]:
        try:
            response = await client.chat.completions.create(
                model="solar-pro",
                messages=messages
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"\n❌ API 호출 오류: {e} → 입력 문장: {input_text[:50]}...")
            return "[ERROR]"

# 📊 전체 평가 함수
async def evaluate_all_concurrent(batch_size=100):
    results = []
    success_count = 0
    error_count = 0

    async def worker(row):
        nonlocal success_count, error_count
        client = next(client_cycle)
        messages = format_prompt(template_str, row["input"])
        pred = await call_with_limit(client, messages, row["input"])
        if pred == "[ERROR]":
            error_count += 1
        else:
            success_count += 1
        return {"prediction": pred, "output": row["output"]}

    for i in tqdm(range(0, len(sample), batch_size), desc="🔁 전체 진행 상황"):
        batch = sample[i:i+batch_size]
        tasks = [worker(row) for row in batch]
        batch_results = await asyncio.gather(*tasks)
        results.extend(batch_results)

    # 🔍 점수 계산
    recalls = []
    for res in results:
        _, recall, _ = compute_scores(res["prediction"], res["output"])
        recalls.append(recall)

    avg_recall = sum(recalls) / len(recalls) if recalls else 0.0
    print(f"\n📊 [{selected_template_id}] 평균 Recall: {round(avg_recall, 4)}")
    print(f"✅ 성공 요청 수: {success_count}개")
    print(f"⚠️ 실패 요청 수: {error_count}개")

# ▶ 실행
await evaluate_all_concurrent(batch_size=100)

🔁 전체 진행 상황: 100%|██████████| 109/109 [25:00<00:00, 13.76s/it]



📊 [base_02] 평균 Recall: 0.9768
✅ 성공 요청 수: 10805개
⚠️ 실패 요청 수: 0개


In [2]:
import asyncio
import pandas as pd
import json
from tqdm import tqdm
from itertools import cycle
from prompts.template_format import format_prompt
from openai import AsyncOpenAI
from dotenv import load_dotenv
import nest_asyncio
import os

# 비동기 루프 적용 (Jupyter용)
nest_asyncio.apply()
load_dotenv()

# API 키 로딩
API_KEYS = [os.getenv(f"UPSTAGE_API_KEY_{i}") for i in range(1, 11)]
API_KEYS = [k for k in API_KEYS if k]

clients = [
    AsyncOpenAI(api_key=key, base_url="https://api.upstage.ai/v1")
    for key in API_KEYS
]
client_cycle = cycle(clients)
semaphores = {client: asyncio.Semaphore(3) for client in clients}  # 키당 최대 동시 3건 제한

# 템플릿 로드
graph_path = "data/prompt_graph.jsonl"
selected_template_id = "base_02"
with open(graph_path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        if obj.get("id") == selected_template_id:
            template_str = obj["template"]
            break

# 테스트 데이터 로딩
df = pd.read_csv("data/test.csv")
sample = [
    {"id": row["id"], "input": row["err_sentence"]}
    for _, row in df.iterrows()
    if pd.notnull(row["err_sentence"])
]

# LLM 호출
async def call_with_limit(client, messages, input_text):
    async with semaphores[client]:
        try:
            response = await client.chat.completions.create(
                model="solar-pro",
                messages=messages
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"\n❌ API 호출 오류: {e} → 입력 문장: {input_text[:50]}...")
            return "[ERROR]"

# 예측 및 저장
async def generate_submission(output_path="data/submission.csv", batch_size=100):
    results = []

    async def worker(row):
        client = next(client_cycle)
        messages = format_prompt(template_str, row["input"])
        pred = await call_with_limit(client, messages, row["input"])
        return {
            "id": row["id"],
            "err_sentence": row["input"],
            "cor_sentence": pred
        }

    for i in tqdm(range(0, len(sample), batch_size), desc="📤 테스트셋 예측 진행 중"):
        batch = sample[i:i+batch_size]
        tasks = [worker(row) for row in batch]
        batch_results = await asyncio.gather(*tasks)
        results.extend(batch_results)

    # 저장
    submission_df = pd.DataFrame(results)
    submission_df.to_csv(output_path, index=False)
    print(f"\n✅ 예측 완료: {len(results)}개 문장 → '{output_path}' 저장됨")

# ▶ 실행
await generate_submission()

📤 테스트셋 예측 진행 중: 100%|██████████| 109/109 [31:17<00:00, 17.23s/it]


✅ 예측 완료: 10870개 문장 → 'data/submission.csv' 저장됨
